In [9]:
import numpy as np
import plotly.graph_objects as go
import scipy
import os

# loaded directory
directory = 'outputs/polycam_mate_floor/depth-nerfacto/2023-11-22_181036/wandb/plots'

In [29]:
# Load the occupancy data from the .npy file
occupancy_grid = np.load(os.path.join(directory, 'object_occupancy.npy'))
# occupancy_grid = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\object_occupancy.npy')
#G:\Mixed_Reality\cs\object_occupancy.npy
# Get the dimensions of the grid
grid_size = occupancy_grid.shape[0]

# Generate grid coordinates
x, y, z = np.meshgrid(np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size),
                      indexing='ij')

# Filter coordinates based on occupancy
occupied_x = x[occupancy_grid]
occupied_y = y[occupancy_grid]
occupied_z = z[occupancy_grid]
print(np.min(occupied_x), np.max(occupied_x))
print(np.min(occupied_y), np.max(occupied_y))
print(np.min(occupied_z), np.max(occupied_z))

# Create a 3D scatter plot
occupancy = go.Scatter3d(x=occupied_x, y=occupied_y, z=occupied_z, mode='markers', marker=dict(size=1))
fig = go.Figure(data=[occupancy])

# Update layout
fig.update_layout(
    title="3D Occupancy Grid",
    scene=dict(
        xaxis=dict(nticks=4, range=[-1, 1]),
        yaxis=dict(nticks=4, range=[-1, 1]),
        zaxis=dict(nticks=4, range=[-1, 1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


-0.07086614173228345 0.055118110236220375
-0.18110236220472442 -0.07086614173228345
-0.8740157480314961 -0.5748031496062992


In [31]:
# Points for plane fitting
points = np.load(os.path.join(directory, 'plane_samples.npy'))
# points = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\plane_samples.npy')


# Plane equation coefficients
coeffs = np.load(os.path.join(directory, 'plane_coefficients.npy'))
# coeffs = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\plane_coefficients.npy')
a, b, c, d = coeffs

# Bounding box vertices
# bbox_vertices = np.array([
#     [ 0.0114, -0.1916, -0.0216], [-0.5252, -0.1620, -0.3944],
#     [ 0.2723, -0.1808, -0.3962], [-0.2642, -0.1512, -0.7690],
#     [ 0.0221, 0.2614, -0.0010], [-0.5144, 0.2910, -0.3738],
#     [ 0.2830, 0.2722, -0.3756], [-0.2535, 0.3019, -0.7485]
# ])

# Load the AABB from the .npy file
aabb = np.load(os.path.join(directory, 'aabb.npy'))
# aabb = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\aabb.npy')
min_corner = aabb[0]
max_corner = aabb[1]
# Compute the vertices of the AABB
vertices = np.array([
    [min_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], max_corner[1], max_corner[2]],
    [min_corner[0], max_corner[1], max_corner[2]],
])

# Define the edges of the AABB
edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],  # bottom edges
    [4, 5], [5, 6], [6, 7], [7, 4],  # top edges
    [0, 4], [1, 5], [2, 6], [3, 7],  # side edges
]

# Plot the edges
edge_traces = [
    go.Scatter3d(
        x=[vertices[pair[0], 0], vertices[pair[1], 0]],
        y=[vertices[pair[0], 1], vertices[pair[1], 1]],
        z=[vertices[pair[0], 2], vertices[pair[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2)
    ) for pair in edges
]

# Create a grid for the plane
xx, yy = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
zz = (-a * xx - b * yy - d) / c

x_values = []
y_values = []
z_values = []
offset = 0
# print(points)

""" for point in points:
    if offset % 200 == 0:
        x_values.append(point[0])
        y_values.append(point[1])
        z_values.append(point[2])
    offset += 1

scatter = go.Scatter3d(x=x_values, y=y_values, z=z_values, mode='markers', marker=dict(size=5)) """

# read "sample_colors.npy"
rgb = np.load(os.path.join(directory, 'sample_colors.npy')) # (N, 3)
# find the median of each column
median = np.median(rgb, axis=0) # (3,)
# find the covariance matrix of the rgb values
cov = np.cov(rgb.T) # (3, 3)
# inverse of the covariance matrix
cov_inv = np.linalg.inv(cov) # (3, 3)
# find the mahalanobis distance of each point from the median
mahalanobis = scipy.spatial.distance.cdist(rgb, [median], metric='mahalanobis', VI=cov_inv) # (N, 1)
mahalanobis_similarity = 1 / (1 + mahalanobis) # (N, 1)

# points = points[mahalanobis_similarity[:, 0] > 0.6]
# print(f"The number of points after color filtering is {len(points)}")

scatter = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=2, color=rgb))

# Plot plane
plane = go.Surface(x=xx, y=yy, z=zz, opacity=0.5)

# Plot bounding box
# bbox_edges = [
#     go.Scatter3d(x=bbox_vertices[[i, j], 0], y=bbox_vertices[[i, j], 1], z=bbox_vertices[[i, j], 2], mode='lines', line=dict(color='red'))
#     for i in range(len(bbox_vertices)) for j in range(i+1, len(bbox_vertices))
# ]

# plot aabb (axis aligned bounding box)


# Combine all plots
fig = go.Figure(data=[scatter, plane, occupancy] + edge_traces)
# fig = go.Figure(data=[scatter, occupancy] + edge_traces)

# Update layout
fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[-1,1]),
        yaxis=dict(nticks=4, range=[-1,1]),
        zaxis=dict(nticks=4, range=[-1,1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


In [32]:
# # read "sample_colors.npy"
# rgb = np.load('sample_colors.npy') # (N, 3)

# # find the median of each column
# median = np.median(rgb, axis=0) # (3,)
# # find the covariance matrix of the rgb values
# cov = np.cov(rgb.T) # (3, 3)
# # inverse of the covariance matrix
# cov_inv = np.linalg.inv(cov) # (3, 3)
# # find the mahalanobis distance of each point from the median
# mahalanobis = scipy.spatial.distance.cdist(rgb, [median], metric='mahalanobis', VI=cov_inv) # (N, 1)
# mahalanobis_similarity = 1 / (1 + mahalanobis) # (N, 1)

# plot the rgb values of all N points, while making the color of each point the corresponding rgb value
table_rgb_scatter = go.Scatter3d(x=rgb[:,0], y=rgb[:,1], z=rgb[:,2], mode='markers', marker=dict(size=2, color=rgb))

# plot the median rgb value as a red point
median_rgb_scatter = go.Scatter3d(x=[median[0]], y=[median[1]], z=[median[2]], mode='markers', marker=dict(size=5, color='red'))

# plot the sample points with real colors
scatter_colored = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=5, color=rgb))

# show the plot
table_rgb_fig = go.Figure(data=[table_rgb_scatter, median_rgb_scatter])
# table_rgb_fig = go.Figure(data=[table_rgb_scatter])

# set axis labels as R, G, B, and axis ranges as [0, 1]
table_rgb_fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[0,1]),
        yaxis=dict(nticks=4, range=[0,1]),
        zaxis=dict(nticks=4, range=[0,1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)


table_rgb_fig.show()
